# THE ANALYSIS OF COVID-19 VACCINE  GLOBAL DISTRIBUTION


In [1]:
# Import pyspark libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import*

In [2]:
# Initialize the spark session

spark = SparkSession.builder.appName("COVID-19 Vaccination Progress Analaysis").getOrCreate()

In [3]:
# Improve PySpark DataFrame.show output in Jupyter notebook

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [4]:
# Reading the data from storage in spark dataframe with its original schema(inferSchema=True)

cvac_df = spark.read.csv("gs://arcane-pillar-307322/data/country_vaccinations.csv", header=True, inferSchema=True)


In [5]:
cvac_df

country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
Afghanistan,AFG,2021-02-22,0.0,0.0,null,null,null,0.0,0.0,null,null,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-02-23,null,null,null,null,1367.0,null,null,null,35.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-02-24,null,null,null,null,1367.0,null,null,null,35.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-02-25,null,null,null,null,1367.0,null,null,null,35.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-02-26,null,null,null,null,1367.0,null,null,null,35.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-02-27,null,null,null,null,1367.0,null,null,null,35.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-02-28,8200.0,8200.0,null,null,1367.0,0.02,0.02,null,35.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-03-01,null,null,null,null,1580.0,null,null,null,41.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-03-02,null,null,null,null,1794.0,null,null,null,46.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...
Afghanistan,AFG,2021-03-03,null,null,null,null,2008.0,null,null,null,52.0,Oxford/AstraZeneca,Government of Afg...,http://www.xinhua...


In [6]:
# Total number of rows in the data frame
cvac_df.count()

9576

In [7]:
# check how spark stores the schema 

cvac_df.printSchema()

root
 |-- country: string (nullable = true)
 |-- iso_code: string (nullable = true)
 |-- date: string (nullable = true)
 |-- total_vaccinations: double (nullable = true)
 |-- people_vaccinated: double (nullable = true)
 |-- people_fully_vaccinated: double (nullable = true)
 |-- daily_vaccinations_raw: double (nullable = true)
 |-- daily_vaccinations: double (nullable = true)
 |-- total_vaccinations_per_hundred: double (nullable = true)
 |-- people_vaccinated_per_hundred: double (nullable = true)
 |-- people_fully_vaccinated_per_hundred: double (nullable = true)
 |-- daily_vaccinations_per_million: double (nullable = true)
 |-- vaccines: string (nullable = true)
 |-- source_name: string (nullable = true)
 |-- source_website: string (nullable = true)



### Data cleaning

###### Droping the irrelavent columns for our analysis

In [8]:
# Drop daily vaccination raw and source website columns

drop_column_list = ["daily_vaccinations_raw","source_website" ]
cvac_df = cvac_df.select([column for column in cvac_df.columns if column not in drop_column_list])


###### Count the null values for each column in the dataframe

In [9]:
# Count the null values in each column using pyspark.sql.functions isnull, when, count

cvac_df.select([count(when(isnull(column), column)).alias(column) for column in cvac_df.columns])


country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name
0,0,0,3804,4409,5968,183,3804,4409,5968,183,0,0


###### Dataframe has many null values in most of the columns, Replace the null values with 0 for further analysis

In [10]:
# Replace null values with 0 

cvac_df = cvac_df.na.fill(value=0)
cvac_df.select('date','total_vaccinations','people_vaccinated','people_fully_vaccinated').show(3,False)

+----------+------------------+-----------------+-----------------------+
|date      |total_vaccinations|people_vaccinated|people_fully_vaccinated|
+----------+------------------+-----------------+-----------------------+
|2021-02-22|0.0               |0.0              |0.0                    |
|2021-02-23|0.0               |0.0              |0.0                    |
|2021-02-24|0.0               |0.0              |0.0                    |
+----------+------------------+-----------------+-----------------------+
only showing top 3 rows



###### Change the data types of the columns 

In [11]:
# Change the date type to the columns 

cvac_df = cvac_df.withColumn("date",cvac_df['date'].cast(DateType()))
cvac_df = cvac_df.withColumn("total_vaccinations",cvac_df['total_vaccinations'].cast(IntegerType()))
cvac_df = cvac_df.withColumn("people_vaccinated",cvac_df['people_vaccinated'].cast(IntegerType()))
cvac_df = cvac_df.withColumn("people_fully_vaccinated",cvac_df['people_fully_vaccinated'].cast(IntegerType()))
cvac_df = cvac_df.withColumn("daily_vaccinations",cvac_df['daily_vaccinations'].cast(IntegerType()))
cvac_df = cvac_df.withColumn("total_vaccinations_per_hundred",cvac_df['total_vaccinations_per_hundred'].cast(IntegerType()))
cvac_df = cvac_df.withColumn("people_vaccinated_per_hundred",cvac_df['people_vaccinated_per_hundred'].cast(IntegerType()))
cvac_df = cvac_df.withColumn("people_fully_vaccinated_per_hundred",cvac_df['people_fully_vaccinated_per_hundred'].cast(IntegerType()))
cvac_df = cvac_df.withColumn("daily_vaccinations_per_million",cvac_df['daily_vaccinations_per_million'].cast(IntegerType()))




cvac_df.select('date','total_vaccinations','people_vaccinated','people_fully_vaccinated').show(5,False)

+----------+------------------+-----------------+-----------------------+
|date      |total_vaccinations|people_vaccinated|people_fully_vaccinated|
+----------+------------------+-----------------+-----------------------+
|2021-02-22|0                 |0                |0                      |
|2021-02-23|0                 |0                |0                      |
|2021-02-24|0                 |0                |0                      |
|2021-02-25|0                 |0                |0                      |
|2021-02-26|0                 |0                |0                      |
+----------+------------------+-----------------+-----------------------+
only showing top 5 rows



In [12]:
# dataset has COVID-19 vaccine data for 166 countries
cvac_df.select('country').distinct().count()

166

In [13]:
# check the dataframe

cvac_df

country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name
Afghanistan,AFG,2021-02-22,0,0,0,0,0,0,0,0,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-02-23,0,0,0,1367,0,0,0,35,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-02-24,0,0,0,1367,0,0,0,35,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-02-25,0,0,0,1367,0,0,0,35,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-02-26,0,0,0,1367,0,0,0,35,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-02-27,0,0,0,1367,0,0,0,35,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-02-28,8200,8200,0,1367,0,0,0,35,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-03-01,0,0,0,1580,0,0,0,41,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-03-02,0,0,0,1794,0,0,0,46,Oxford/AstraZeneca,Government of Afg...
Afghanistan,AFG,2021-03-03,0,0,0,2008,0,0,0,52,Oxford/AstraZeneca,Government of Afg...


## Vaccine distribution analysis using sparkSQL

###  10 Countries with the maximum number of total vaccination



In [14]:
# 10 Countries with the maximum number of total vaccination 

vac_count = cvac_df.groupBy("country").max("total_vaccinations")
vac_count = vac_count.orderBy('max(total_vaccinations)', ascending=False)
vac_count.show(10,False) 

+--------------+-----------------------+
|country       |max(total_vaccinations)|
+--------------+-----------------------+
|United States |161688422              |
|China         |133801000              |
|India         |75979651               |
|United Kingdom|36631187               |
|England       |30791767               |
|Brazil        |20956770               |
|Turkey        |16602919               |
|Germany       |14374088               |
|Indonesia     |12292491               |
|France        |12139523               |
+--------------+-----------------------+
only showing top 10 rows



In [15]:
# saving the spark output in partitions
# vac_count.write.csv("gs://arcane-pillar-307322/data/vaccines_counts.csv", header=True)

# save the spark output in one csv file
vac_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/vaccines_maxcounts")

### The total number of people vaccinated in each countries


In [16]:
# The Total number of people vaccinated in each countries, we will check top 10 countries 

vac_peoplecount = cvac_df.groupBy("country").max("people_vaccinated")
vac_peoplecount = vac_peoplecount.orderBy('max(people_vaccinated)', ascending=False)
vac_peoplecount.show(10,False) 

+--------------+----------------------+
|country       |max(people_vaccinated)|
+--------------+----------------------+
|United States |104213478             |
|India         |65739470              |
|United Kingdom|31425682              |
|England       |26644910              |
|Brazil        |16377083              |
|Germany       |10039938              |
|Turkey        |9478896               |
|France        |9109776               |
|Indonesia     |8424729               |
|Mexico        |7903660               |
+--------------+----------------------+
only showing top 10 rows



In [17]:
# save the spark output in one csv file

vac_peoplecount.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/total_people_vaccinated")

### The total number of people fully vaccinated in each countries


In [18]:
# The Total number of people fully vaccinated in each countries, we will check top 10 countries 

vac_peopleFcount = cvac_df.groupBy("country").max("people_fully_vaccinated")
vac_peopleFcount = vac_peopleFcount.orderBy('max(people_fully_vaccinated)', ascending=False)
vac_peopleFcount.show(10,False) 

+--------------+----------------------------+
|country       |max(people_fully_vaccinated)|
+--------------+----------------------------+
|United States |59858146                    |
|India         |10240181                    |
|Turkey        |7124023                     |
|United Kingdom|5205505                     |
|Brazil        |5078476                     |
|Israel        |4815283                     |
|Russia        |4593824                     |
|Germany       |4334150                     |
|England       |4146857                     |
|Morocco       |3891875                     |
+--------------+----------------------------+
only showing top 10 rows



In [19]:
# save the spark output in one csv file

vac_peopleFcount.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/people_fully_vaccinated")

### Analysis of people_vaccinated and people_fully_vaccinated

In [20]:
# Data analysis of people_vaccinated and people_full_vaccinated

counts_vaccs = cvac_df.groupBy('country').max('people_vaccinated','people_fully_vaccinated')
counts_vaccs = counts_vaccs.orderBy('max(people_vaccinated)','max(people_fully_vaccinated)', ascending=False)
counts_vaccs.show(10)

+--------------+----------------------+----------------------------+
|       country|max(people_vaccinated)|max(people_fully_vaccinated)|
+--------------+----------------------+----------------------------+
| United States|             104213478|                    59858146|
|         India|              65739470|                    10240181|
|United Kingdom|              31425682|                     5205505|
|       England|              26644910|                     4146857|
|        Brazil|              16377083|                     5078476|
|       Germany|              10039938|                     4334150|
|        Turkey|               9478896|                     7124023|
|        France|               9109776|                     3029747|
|     Indonesia|               8424729|                     3867762|
|        Mexico|               7903660|                     1083530|
+--------------+----------------------+----------------------------+
only showing top 10 rows



In [21]:
# save the spark output in one csv file

counts_vaccs.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/total_people_vaccinated_vs_fully_vaccinated")

### The number of daily vaccinations in each countries


In [22]:
# The Total number of daily vaccinations in each countries, we will check top 10 countries 

vacdaily_count = cvac_df.groupBy("country").max("daily_vaccinations")
vacdaily_count = vacdaily_count.orderBy('max(daily_vaccinations)', ascending=False)
vacdaily_count.show(10)

+--------------+-----------------------+
|       country|max(daily_vaccinations)|
+--------------+-----------------------+
|         China|                5190143|
| United States|                3072527|
|         India|                2309232|
|United Kingdom|                 602266|
|        Brazil|                 592115|
|       England|                 507875|
|     Indonesia|                 449308|
|        Turkey|                 435596|
|        Russia|                 345222|
|        Mexico|                 323200|
+--------------+-----------------------+
only showing top 10 rows



In [23]:
# save the spark output in one csv file

vacdaily_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/daily_vaccinations")

### Total vaccinations per hundred in each country

In [24]:
# total_vaccinations_per_hundred

vac_maxperhundred=cvac_df.groupBy("country").max("total_vaccinations_per_hundred")
vac_maxperhundred=vac_maxperhundred.orderBy('max(total_vaccinations_per_hundred)',ascending=False)
vac_maxperhundred.show()

+--------------------+-----------------------------------+
|             country|max(total_vaccinations_per_hundred)|
+--------------------+-----------------------------------+
|           Gibraltar|                                180|
|              Israel|                                116|
|          Seychelles|                                103|
|United Arab Emirates|                                 86|
|      Cayman Islands|                                 77|
|         Isle of Man|                                 63|
|    Falkland Islands|                                 62|
|               Wales|                                 61|
|             Bermuda|                                 60|
|              Bhutan|                                 59|
|              Jersey|                                 58|
|               Chile|                                 56|
|            Scotland|                                 54|
|             England|                                 5

In [25]:
# save the spark output in one csv file

vac_maxperhundred.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/total_vaccine_per_hundred")

### Total people vaccinated per hundred in each country


In [26]:
# people_vaccinated_per_hundred

vacPH_count = cvac_df.groupBy("country").max("people_vaccinated_per_hundred")
vacPH_count = vacPH_count.orderBy('max(people_vaccinated_per_hundred)', ascending=False)
vacPH_count.show(10,False) 

+----------------+----------------------------------+
|country         |max(people_vaccinated_per_hundred)|
+----------------+----------------------------------+
|Gibraltar       |95                                |
|Seychelles      |65                                |
|Falkland Islands|62                                |
|Israel          |60                                |
|Bhutan          |59                                |
|Saint Helena    |51                                |
|England         |47                                |
|Scotland        |46                                |
|Cayman Islands  |46                                |
|United Kingdom  |46                                |
+----------------+----------------------------------+
only showing top 10 rows



In [27]:
# save the spark output in one csv file

vacPH_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/total_people_vaccinated_hundred")

### People Fully vaccinated per hundred in each country

In [28]:
# people_fully_vaccinated_per_hundred

vacFPH_count = cvac_df.groupBy("country").max("people_fully_vaccinated_per_hundred")
vacFPH_count = vacFPH_count.orderBy('max(people_fully_vaccinated_per_hundred)', ascending=False)
vacFPH_count.show(10,False)

+--------------------+----------------------------------------+
|country             |max(people_fully_vaccinated_per_hundred)|
+--------------------+----------------------------------------+
|Gibraltar           |84                                      |
|Israel              |55                                      |
|Seychelles          |38                                      |
|Cayman Islands      |30                                      |
|Bermuda             |24                                      |
|United Arab Emirates|22                                      |
|Monaco              |21                                      |
|Chile               |20                                      |
|Isle of Man         |18                                      |
|United States       |17                                      |
+--------------------+----------------------------------------+
only showing top 10 rows



In [29]:
# save the spark output in one csv file

vacFPH_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/people_fully_vaccinated_hundred")

### # Data analysis of people vaccinated per hundred, and people fully vaccinated per hundred


In [30]:
# Data analysis of people vaccinated per hundred and people fully vaccinated per hundred

counts_ph = cvac_df.groupBy('country').max('people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred')
counts_ph = counts_ph.orderBy('max(people_vaccinated_per_hundred)','max(people_fully_vaccinated_per_hundred)', ascending=False)
counts_ph.show(5)

+----------------+----------------------------------+----------------------------------------+
|         country|max(people_vaccinated_per_hundred)|max(people_fully_vaccinated_per_hundred)|
+----------------+----------------------------------+----------------------------------------+
|       Gibraltar|                                95|                                      84|
|      Seychelles|                                65|                                      38|
|Falkland Islands|                                62|                                       0|
|          Israel|                                60|                                      55|
|          Bhutan|                                59|                                       0|
+----------------+----------------------------------+----------------------------------------+
only showing top 5 rows



In [31]:
# save the spark output in one csv file

counts_ph.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/vaccine_analysis_per_hundred")

### Daily vaccinations per million in each country

In [32]:
# daily_vaccinations_per_million

vacDM_count = cvac_df.groupBy("country").max("daily_vaccinations_per_million")
vacDM_count = vacDM_count.orderBy('max(daily_vaccinations_per_million)', ascending=False)
vacDM_count.show(10,False) 

+--------------------+-----------------------------------+
|country             |max(daily_vaccinations_per_million)|
+--------------------+-----------------------------------+
|Bhutan              |118759                             |
|Falkland Islands    |54264                              |
|Gibraltar           |31700                              |
|Seychelles          |24415                              |
|Anguilla            |22064                              |
|Israel              |21362                              |
|Isle of Man         |18240                              |
|Bahamas             |17521                              |
|United Arab Emirates|15703                              |
|Chile               |15584                              |
+--------------------+-----------------------------------+
only showing top 10 rows



In [33]:
# save the spark output in one csv file

vacDM_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/daily_vaccination_per_million")

## Vaccine Analaysis

In [34]:
# importing library for sql functions

import pyspark.sql.functions as F

In [35]:
# List of all the vaccines
cvac_df.select('vaccines').distinct().collect()

[Row(vaccines='Oxford/AstraZeneca'),
 Row(vaccines='EpiVacCorona, Sputnik V'),
 Row(vaccines='Pfizer/BioNTech, Sinovac, Sputnik V'),
 Row(vaccines='Sinovac'),
 Row(vaccines='Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V'),
 Row(vaccines='Oxford/AstraZeneca, Sinopharm/Beijing'),
 Row(vaccines='Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V'),
 Row(vaccines='Moderna, Oxford/AstraZeneca'),
 Row(vaccines='Johnson&Johnson'),
 Row(vaccines='Sinopharm/Beijing, Sinopharm/Wuhan, Sinovac'),
 Row(vaccines='Oxford/AstraZeneca, Pfizer/BioNTech, Sputnik V'),
 Row(vaccines='Moderna, Oxford/AstraZeneca, Pfizer/BioNTech'),
 Row(vaccines='Sinopharm/Beijing, Sputnik V'),
 Row(vaccines='Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinopharm/Wuhan, Sputnik V'),
 Row(vaccines='Sputnik V'),
 Row(vaccines='Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V'),
 Row(vaccines='Moderna'),
 Row(vaccines='Moderna, Pfizer/BioNTech'),
 Row(vaccines='Pfizer/BioN

##  List of vaccines used by the countries


In [36]:
# List of vaccines used by the countries

cvacs_df = cvac_df.withColumn('vaccines', F.explode(F.array('vaccines'))).groupby('country').agg(F.collect_set('vaccines').alias('vaccines'))
cvacs_df.show(20,0)

+-----------+----------------------------------------------+
|country    |vaccines                                      |
+-----------+----------------------------------------------+
|Anguilla   |[Oxford/AstraZeneca]                          |
|Paraguay   |[Sputnik V]                                   |
|Russia     |[EpiVacCorona, Sputnik V]                     |
|Macao      |[Pfizer/BioNTech, Sinopharm/Beijing]          |
|Senegal    |[Sinopharm/Beijing]                           |
|Sweden     |[Oxford/AstraZeneca, Pfizer/BioNTech]         |
|Guyana     |[Oxford/AstraZeneca]                          |
|Jersey     |[Oxford/AstraZeneca, Pfizer/BioNTech]         |
|Philippines|[Oxford/AstraZeneca, Sinovac]                 |
|Malaysia   |[Pfizer/BioNTech, Sinovac]                    |
|Singapore  |[Moderna, Pfizer/BioNTech]                    |
|Turkey     |[Pfizer/BioNTech, Sinovac]                    |
|Malawi     |[Oxford/AstraZeneca]                          |
|Iraq       |[Oxford/Ast

In [37]:
# popular vaccines

vac_names=cvac_df.groupBy("vaccines").count()
vac_names.show()

+--------------------+-----+
|            vaccines|count|
+--------------------+-----+
|  Oxford/AstraZeneca| 1331|
|EpiVacCorona, Spu...|  110|
|Pfizer/BioNTech, ...|   22|
|             Sinovac|   77|
|Oxford/AstraZenec...|  154|
|Oxford/AstraZenec...|  178|
|Oxford/AstraZenec...|  187|
|Moderna, Oxford/A...|   22|
|     Johnson&Johnson|   45|
|Sinopharm/Beijing...|  109|
|Oxford/AstraZenec...|  101|
|Moderna, Oxford/A...| 2255|
|Sinopharm/Beijing...|   44|
|Oxford/AstraZenec...|   89|
|           Sputnik V|  388|
|Moderna, Oxford/A...|   96|
|             Moderna|   37|
|Moderna, Pfizer/B...|  370|
|     Pfizer/BioNTech| 1435|
|Oxford/AstraZenec...|  268|
+--------------------+-----+
only showing top 20 rows



In [38]:
# save the spark output in one csv file

vac_names.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/popularVaccines")

### Daily vaccination distribution by date


In [39]:
cvac_df.select('date').agg(max('date'))

max(date)
2021-04-03


In [40]:
cvac_df.select('date').agg(min('date'))

min(date)
2020-12-13


In [41]:
# Daily vaccination distribution by date
cvac_dvdates = cvac_df.select('date','daily_vaccinations')
cvac_dvdates.show(10)

+----------+------------------+
|      date|daily_vaccinations|
+----------+------------------+
|2021-02-22|                 0|
|2021-02-23|              1367|
|2021-02-24|              1367|
|2021-02-25|              1367|
|2021-02-26|              1367|
|2021-02-27|              1367|
|2021-02-28|              1367|
|2021-03-01|              1580|
|2021-03-02|              1794|
|2021-03-03|              2008|
+----------+------------------+
only showing top 10 rows



In [42]:
# save the spark output in one csv file

cvac_dvdates.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/daily_vaccination_datewise")

In [43]:
# maximum number of daily vaccination group by country and date

vac_maxday=cvac_df.groupBy("country","date").max("daily_vaccinations")
vac_maxday.show()

+-------------------+----------+-----------------------+
|            country|      date|max(daily_vaccinations)|
+-------------------+----------+-----------------------+
|            Algeria|2021-02-01|                   2509|
|Antigua and Barbuda|2021-03-16|                    821|
|          Australia|2021-03-26|                  31343|
|         Bangladesh|2021-03-31|                  43055|
|            Bermuda|2021-02-25|                    631|
|             Brazil|2021-02-09|                 218237|
|           Bulgaria|2021-01-06|                    472|
|           Bulgaria|2021-03-16|                   6263|
|     Cayman Islands|2021-03-24|                    840|
|              Chile|2021-03-06|                 180343|
|            Denmark|2021-02-09|                   9494|
| Dominican Republic|2021-03-20|                  16089|
|             Gambia|2021-03-23|                    281|
|          Gibraltar|2021-02-09|                    804|
|           Honduras|2021-03-10

In [44]:
# save the spark output in one csv file

vac_maxday.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/maximum_daily_vaccination_Countrydatewise")

In [45]:
# people_vaccinated and people_fully_vaccinated distribution by date

pt_date = cvac_df.select('date','people_vaccinated','people_fully_vaccinated')
pt_date.show(10)

+----------+-----------------+-----------------------+
|      date|people_vaccinated|people_fully_vaccinated|
+----------+-----------------+-----------------------+
|2021-02-22|                0|                      0|
|2021-02-23|                0|                      0|
|2021-02-24|                0|                      0|
|2021-02-25|                0|                      0|
|2021-02-26|                0|                      0|
|2021-02-27|                0|                      0|
|2021-02-28|             8200|                      0|
|2021-03-01|                0|                      0|
|2021-03-02|                0|                      0|
|2021-03-03|                0|                      0|
+----------+-----------------+-----------------------+
only showing top 10 rows



In [46]:
# save the spark output in one csv file

pt_date.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/vaccination_vs_fullyV_datewise")

## Yearly analysis of vaccine distribution

In [47]:
# create a column 'year' from the dataset's date column using year function

year = cvac_df.withColumn('year',year(cvac_df.date))


## Vaccine distribution on total_vaccinations and daily_vaccinations year-wise


In [48]:
# vaccine distribution on total_vaccinations and daily_vaccinations year-wise

vacyear_count = year.groupBy('year').max("total_vaccinations", "daily_vaccinations")
vacyear_count = vacyear_count.orderBy('max(total_vaccinations)', 'max(daily_vaccinations)', ascending=False)
vacyear_count.show(10,False) 

+----+-----------------------+-----------------------+
|year|max(total_vaccinations)|max(daily_vaccinations)|
+----+-----------------------+-----------------------+
|2021|161688422              |5190143                |
|2020|4500000                |278776                 |
+----+-----------------------+-----------------------+



In [49]:
# save the spark output in one csv file

vacyear_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/total_daily_vaccination_yearwise")

##  Vaccine distribution on people_vaccinated and people_fully_vaccinated year-wise



In [50]:
# vaccine distribution on people_vaccinated and people_fully_vaccinated year-wise

tpyear_count = year.groupBy('year').max('people_vaccinated','people_fully_vaccinated')
tpyear_count = tpyear_count.orderBy('max(people_vaccinated)','max(people_fully_vaccinated)', ascending=False)
tpyear_count.show(10, False)

+----+----------------------+----------------------------+
|year|max(people_vaccinated)|max(people_fully_vaccinated)|
+----+----------------------+----------------------------+
|2021|104213478             |59858146                    |
|2020|2794588               |847                         |
+----+----------------------+----------------------------+



In [51]:
# save the spark output in one csv file

tpyear_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/peoplevac_peopleFC_yearwise")

## Monthly analysis of vaccine distribution

In [52]:
# create a column 'month' from the dataset's date column using month function

month = cvac_df.withColumn('month',month(cvac_df.date))

## Vaccine distribution on total_vaccinations and daily_vaccinations month-wise


In [53]:
# vaccine distribution on total_vaccinations and daily_vaccinations month-wise

vacmonth_count = month.groupBy('month').max("total_vaccinations", "daily_vaccinations")
vacmonth_count = vacmonth_count.orderBy('max(total_vaccinations)', 'max(daily_vaccinations)', ascending=False)
vacmonth_count.show(10,False) 

+-----+-----------------------+-----------------------+
|month|max(total_vaccinations)|max(daily_vaccinations)|
+-----+-----------------------+-----------------------+
|4    |161688422              |5190143                |
|3    |150273292              |4851614                |
|2    |75236003               |1916190                |
|1    |31123299               |1324949                |
|12   |4500000                |278776                 |
+-----+-----------------------+-----------------------+



In [54]:
# save the spark output in one csv file

vacmonth_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/total_daily_vaccination_monthwise")

##  Vaccine distribution on people_vaccinated and people_fully_vaccinated month-wise


In [55]:
tpmonth_count = month.groupBy('month').max('people_vaccinated','people_fully_vaccinated')
tpmonth_count = tpmonth_count.orderBy('max(people_vaccinated)','max(people_fully_vaccinated)', ascending=False)
tpmonth_count.show(10, False)

+-----+----------------------+----------------------------+
|month|max(people_vaccinated)|max(people_fully_vaccinated)|
+-----+----------------------+----------------------------+
|4    |104213478             |59858146                    |
|3    |97593290              |54607041                    |
|2    |49772180              |24779920                    |
|1    |25201143              |5657142                     |
|12   |2794588               |847                         |
+-----+----------------------+----------------------------+



In [56]:
# save the spark output in one csv file

tpmonth_count.repartition(1).write.mode ("overwrite").format("csv").option("header", "true").save("gs://arcane-pillar-307322/data/peoplevac_peopleFC_monthwise")